In [ ]:
import os

import pandas as pd
from pyspark.sql import SparkSession

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

CATALOG = "glue_catalog"
ICEBERG_S3_ROOT_PATH = "s3a://blueriver-datalake/iceberg"

os.environ["AWS_PROFILE"] = "blueriver"
spark = (
    SparkSession.builder.appName("")
    .config("spark.sql.defaultCatalog", CATALOG)
    .config(f"spark.sql.catalog.{CATALOG}", "org.apache.iceberg.spark.SparkCatalog")
    .config(f"spark.sql.catalog.{CATALOG}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog")
    .config(f"spark.sql.catalog.{CATALOG}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config(f"spark.sql.catalog.{CATALOG}.warehouse", ICEBERG_S3_ROOT_PATH)
    .config(f"spark.sql.catalog.{CATALOG}.s3.path-style-access", True)
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config(
        "spark.hadoop.fs.s3a.aws.credentials.provider",
        "software.amazon.awssdk.auth.credentials.DefaultCredentialsProvider",
    )
    .config("spark.sql.caseSensitive", True)
    .config("spark.sql.session.timeZone", "UTC")
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
    # .config("spark.driver.extraJavaOptions", "-DsocksProxyHost=localhost -DsocksProxyPort=8001")
    # .config("spark.executor.extraJavaOptions", "-DsocksProxyHost=localhost -DsocksProxyPort=8001")
    .getOrCreate()
)

df1 = spark.createDataFrame([("2022-11-12",)], ["ts"])
df2 = spark.sql("SELECT * FROM store_bronze.tb_lower LIMIT 10")